# 03 - Tool Use Fundamentals

**Give your LLM superpowers!** Learn how to define tools that allow LLMs to interact with the real world.

## Learning Objectives

By the end of this notebook, you will:
- Understand what tool use is and why it matters
- Define tools using JSON Schema
- Write effective tool descriptions
- Parse and handle LLM tool requests

## Table of Contents

1. [What is Tool Use?](#what-is-tool-use)
2. [JSON Schema Basics](#json-schema)
3. [Defining Your First Tool](#first-tool)
4. [Writing Good Tool Descriptions](#descriptions)
5. [Using Our Tool Registry](#registry)
6. [Exercises](#exercises)
7. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
import json
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

from openai import OpenAI
client = OpenAI()

print("Setup complete!")

---
## 1. What is Tool Use? <a id='what-is-tool-use'></a>

**Tool use** (also called **function calling**) allows LLMs to:
- Request external actions
- Access real-time data
- Perform calculations
- Interact with APIs

### The Problem Without Tools

LLMs have limitations:
- Knowledge cutoff date
- Can't access current data
- Prone to calculation errors
- Can't take real actions

In [ ]:
# GUIDED: LLM limitations example

# The LLM might not know current weather
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What's the weather in Tokyo right now?"}]
)

print("Without Tools:")
print(response.choices[0].message.content)

### How Tool Use Works

```
1. You define tools (name, description, parameters)
       ↓
2. LLM receives user request + available tools
       ↓
3. LLM decides if a tool is needed
       ↓
4. LLM outputs: which tool + what arguments
       ↓
5. Your code executes the tool
       ↓
6. Tool result sent back to LLM
       ↓
7. LLM generates final response
```

---
## 2. JSON Schema Basics <a id='json-schema'></a>

Tools are defined using **JSON Schema** - a standard way to describe the structure of JSON data.

### Key Concepts

| Property | Description |
|----------|-------------|
| `type` | Data type (string, number, boolean, object, array) |
| `description` | Human-readable explanation |
| `properties` | For objects: the fields it contains |
| `required` | Which fields are mandatory |
| `enum` | Allowed values for a field |

In [ ]:
# GUIDED: JSON Schema examples

# Simple schema for a search query
search_schema = {
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "The search query"
        },
        "max_results": {
            "type": "integer",
            "description": "Maximum number of results to return"
        }
    },
    "required": ["query"]
}

print("Search Query Schema:")
print(json.dumps(search_schema, indent=2))

In [ ]:
# GUIDED: More complex schema with enum and nested objects

order_schema = {
    "type": "object",
    "properties": {
        "product_id": {
            "type": "string",
            "description": "The product identifier"
        },
        "quantity": {
            "type": "integer",
            "description": "Number of items to order"
        },
        "priority": {
            "type": "string",
            "enum": ["standard", "express", "overnight"],
            "description": "Shipping priority"
        },
        "shipping_address": {
            "type": "object",
            "properties": {
                "street": {"type": "string"},
                "city": {"type": "string"},
                "zip": {"type": "string"}
            },
            "required": ["street", "city", "zip"]
        }
    },
    "required": ["product_id", "quantity"]
}

print("Order Schema:")
print(json.dumps(order_schema, indent=2))

---
## 3. Defining Your First Tool <a id='first-tool'></a>

Let's create a simple calculator tool.

In [ ]:
# GUIDED: Define a calculator tool

# The tool definition for OpenAI
calculator_tool = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "Perform a mathematical calculation. Use this for any math operations.",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 2', '10 * 5')"
                }
            },
            "required": ["expression"]
        }
    }
}

print("Calculator Tool Definition:")
print(json.dumps(calculator_tool, indent=2))

In [ ]:
# GUIDED: The actual function that performs the calculation

def calculate(expression: str) -> str:
    """
    Safely evaluate a mathematical expression.
    
    Parameters
    ----------
    expression : str
        A mathematical expression like '2 + 2' or '10 * 5'
    
    Returns
    -------
    str
        The result of the calculation
    """
    # Only allow safe characters
    allowed = set('0123456789+-*/.() ')
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Test it
print("Testing calculate function:")
print(f"  2 + 2 = {calculate('2 + 2')}")
print(f"  10 * 5 = {calculate('10 * 5')}")
print(f"  100 / 4 = {calculate('100 / 4')}")

In [ ]:
# GUIDED: Use the tool with the LLM

# Send request with tool
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is 157 * 23?"}
    ],
    tools=[calculator_tool],
    tool_choice="auto"  # Let the model decide whether to use the tool
)

# Check if the model wants to use a tool
message = response.choices[0].message

if message.tool_calls:
    tool_call = message.tool_calls[0]
    print("LLM wants to use a tool!")
    print(f"  Tool: {tool_call.function.name}")
    print(f"  Arguments: {tool_call.function.arguments}")
    
    # Execute the tool
    args = json.loads(tool_call.function.arguments)
    result = calculate(args["expression"])
    print(f"  Result: {result}")
else:
    print("LLM responded directly:")
    print(message.content)

---
## 4. Writing Good Tool Descriptions <a id='descriptions'></a>

The LLM decides which tool to use based on descriptions. Good descriptions are crucial!

### Description Best Practices

In [ ]:
# GUIDED: Compare bad vs good tool descriptions

# Bad description - vague
bad_tool = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Search for things",  # Too vague!
        "parameters": {
            "type": "object",
            "properties": {
                "q": {"type": "string"}  # Unclear parameter name
            },
            "required": ["q"]
        }
    }
}

# Good description - clear and helpful
good_tool = {
    "type": "function",
    "function": {
        "name": "web_search",
        "description": "Search the web for current information. Use this when you need up-to-date data, news, or facts that might have changed after your training cutoff.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query. Be specific and include relevant keywords."
                },
                "num_results": {
                    "type": "integer",
                    "description": "Number of results to return (1-10). Default is 5."
                }
            },
            "required": ["query"]
        }
    }
}

print("Bad Tool Description:")
print(f"  Name: {bad_tool['function']['name']}")
print(f"  Description: {bad_tool['function']['description']}")

print("\nGood Tool Description:")
print(f"  Name: {good_tool['function']['name']}")
print(f"  Description: {good_tool['function']['description']}")

### Description Checklist

- **What it does**: Clear action description
- **When to use**: Conditions for using this tool
- **Parameters**: Each parameter explained
- **Examples**: Sample values in descriptions
- **Constraints**: Limits or requirements

---
## 5. Using Our Tool Registry <a id='registry'></a>

Let's use the `ToolRegistry` from our utility modules to manage tools.

In [ ]:
# GUIDED: Import and use the ToolRegistry

from src.tool_registry import Tool, ToolRegistry, create_tool

# Create a registry
registry = ToolRegistry()

# Create tools using our helper
add_tool = create_tool(
    name="add",
    description="Add two numbers together",
    function=lambda a, b: a + b
)

multiply_tool = create_tool(
    name="multiply",
    description="Multiply two numbers",
    function=lambda a, b: a * b
)

# Register tools
registry.register(add_tool)
registry.register(multiply_tool)

print("\nRegistered tools:", registry.list_tools())

In [ ]:
# GUIDED: Use tools from the registry

# Execute a tool
result = registry.execute("add", a=5, b=3)
print(f"add(5, 3) = {result.result}")

result = registry.execute("multiply", a=4, b=7)
print(f"multiply(4, 7) = {result.result}")

# Get tools in OpenAI format
print("\nOpenAI Format:")
print(json.dumps(registry.to_openai_format(), indent=2))

In [ ]:
# GUIDED: Use pre-built calculator tools

from tools.calculator import get_calculator_tools, register_all

# Get a registry with all calculator tools
calc_registry = register_all()

print("\nCalculator Tools:")
print(calc_registry.describe())

---
## 6. Exercises <a id='exercises'></a>

Practice defining and using tools!

### Exercise 1: Define a Weather Tool

Create a tool definition for getting weather information.

In [ ]:
# TODO: Define a weather tool with:
# - location (required): city name
# - units (optional): "celsius" or "fahrenheit"

weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        # Add description and parameters
    }
}

# Your code here:


### Exercise 2: Create a Text Tool

Create a tool that counts words in text.

In [ ]:
# TODO: Create a word_count tool using create_tool
# The function should take text and return the word count

# Your code here:


### Exercise 3: Full Tool Flow

Send a message to the LLM with your calculator tools and handle the response.

In [ ]:
# TODO: Complete the tool flow
# 1. Send "What is 45 plus 67?" to the LLM with calculator tools
# 2. If it requests a tool, execute it using the registry
# 3. Print the result

# Your code here:


---
## 7. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You understand the tool use flow
- [ ] You can define tools using JSON Schema
- [ ] You know what makes a good tool description
- [ ] You can use the ToolRegistry
- [ ] You completed at least 2 exercises

### Next Steps

In the next notebook, we'll dive deeper into **Function Calling** - handling multi-turn conversations with tools and working with both OpenAI and Anthropic formats!

---
## Summary

**Key Concepts:**

1. **Tool Use** lets LLMs access external capabilities
2. **JSON Schema** defines tool parameters
3. **Good Descriptions** help the LLM choose the right tool
4. **ToolRegistry** manages tool definitions and execution

**The Tool Flow:**
```
User Request → LLM + Tools → Tool Call → Execute → Result → LLM → Response
```